# Run Ensemble Mean upon Anemoi Models
Connecting cascade and anemoi allows for distributed and managed execution of models, and generation of products.

In [2]:
import anemoi.cascade as ac

In [3]:
import os

# Setup Environment
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True'
os.environ['LD_LIBRARY_PATH'] = f"INSERT_LD_LIBRARY_PATH_HERE:{os.environ['LD_LIBRARY_PATH']}"

## Additional cascade actions
`pproc-cascade` provides an action api for advanced production generation, but depends on pproc

In [4]:
from cascade import Cascade
import earthkit.data as ekd

PPCASCADE_IMPORTED = True
try:
    from ppcascade import fluent as ppfluent
except (RuntimeError, ImportError, AttributeError) as e:
    print('pproc cascade could not be imported', e)
    PPCASCADE_IMPORTED = False

In [5]:
CKPT = {'huggingface':'ecmwf/aifs-single-1.0'}

With the environment ready, and the ckpt established, we can begin to call a prediction

In [6]:
ac.fluent.from_input?

Signature:
ac.fluent.from_input(
    ckpt: 'VALID_CKPT',
    input: 'str | dict[str, Any]',
    date: 'str | tuple[int, int, int]',
    lead_time: 'Any',
    *,
    ensemble_members: 'int' = 1,
    **kwargs,
) -> 'fluent.Action'
Docstring:
Run an anemoi model from a given input source

Parameters
----------
ckpt : VALID_CKPT
    Checkpoint to load
input : str | dict[str, Any]
    `anemoi.inference` input source.
    Can be `mars`, `grib`, etc or a dictionary of input configuration
date : str | tuple[int, int, int]
    Date to get initial conditions for
lead_time : Any
    Lead time to run out to. Can be a string, 
    i.e. `1H`, `1D`, int, or a datetime.timedelta
ensemble_members : int, optional
    Number of ensemble members to run, by default 1

Returns
-------
fluent.Action
    Cascade action of the model results

Examples
-------
>>> from anemoicascade import from_input
>>> from_input("anemoi_model.ckpt", "mars", date = "2021-01-01T00:00:00", lead_time = "10D")
File:      /etc/ecmw

All that is needed to run a prediction is the:
- checkpoint
- the input_type
- Start time
- Lead time

From there, the number of ensembles can be set, which is automatically added as a dimension

Note:
    Some issues do occur with running out of memory at higher ensemble numbers

In [7]:
model_action = ac.fluent.from_input(CKPT, 'mars', '2022-01-01T00:00', lead_time = '7D', ensemble_members=51)
model_action.nodes

/perm/ecm1947/venvs/cascade/lib/python3.11/site-packages/anemoi/inference/runners/default.py:149: UserWarning: 
            No post_processors defined. Accumulations will be accumulated from the beginning of the forecast.

            🚧🚧🚧 In a future release, the default will be to NOT accumulate from the beginning of the forecast. 🚧🚧🚧
            Update your config if you wish to keep accumulating from the beginning.
            https://github.com/ecmwf/anemoi-inference/issues/131
            
  warnings.warn(
/perm/ecm1947/venvs/cascade/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 12 files: 100%|██████████| 12/12 [00:00<00:00, 34.95it/s]
/perm/ecm1947/venvs/cascade/lib/python3.11/site-packages/anemoi/utils/provenance.py:93: UserWarning: The '__version__' attribute is deprecated and will be

<xarray.Dataset> Size: 344kB
Dimensions:          (param: 30, ensemble_member: 51, date: 1, step: 28)
Coordinates:
  * param            (param) object 240B 'cp' 'tp' '100u' '100v' ... 'u' 't' 'z'
  * ensemble_member  (ensemble_member) int64 408B 0 1 2 3 4 5 ... 46 47 48 49 50
  * date             (date) <U16 64B '2022-01-01T00:00'
  * step             (step) int64 224B 6 12 18 24 30 36 ... 144 150 156 162 168
Data variables:
    default          (param, ensemble_member, date, step) object 343kB Node t...

Now we can check the coordinates of the graph.

In [8]:
model_action.nodes.coords

Coordinates:
  * param            (param) object 240B 'cp' 'tp' '100u' '100v' ... 'u' 't' 'z'
  * ensemble_member  (ensemble_member) int64 408B 0 1 2 3 4 5 ... 46 47 48 49 50
  * date             (date) <U16 64B '2022-01-01T00:00'
  * step             (step) int64 224B 6 12 18 24 30 36 ... 144 150 156 162 168

In [ ]:
if PPCASCADE_IMPORTED:
    from ppcascade.utils.window import Range
    
    interpolation = {
        "grid": "O320"
    }
    
    windows = [
                Range("0-24", [6, 12, 18, 24]), 
                Range("12-24", [12, 18, 24]), 
            ]


In [ ]:
PRODUCT_CATALOG = {
    'ensemble_window_mean': lambda x: x.switch(ppfluent.Action).window_operation("mean", windows,dim="step", batch_size=2).ensemble_operation("mean", dim="ensemble_member"),
    'ensemble_mean': lambda x: x.select({'step':24}).mean(dim="member"),
}

In [ ]:
graph = PRODUCT_CATALOG['ensemble_mean'](model_action)

With the ensemble mean for a bunch of the params chosen, we can visualise the graph

In [ ]:
cascade_sel = Cascade.from_actions([graph])
cascade_sel.visualise("EnsembleMean.html", cdn_resources='in_line', preset='blob')

Now the fun part, execution...

In [ ]:
%%time

from cascade.executors.dask import DaskLocalExecutor 

cascade_sel.executor = DaskLocalExecutor(memory_limit="24GB", n_workers=1, threads_per_worker=1)
results = cascade_sel.execute()

In [ ]:
combined_data = ekd.from_source('multi', list(results.values()))

In [ ]:
combined_data.ls()

## Plotting

In [ ]:
from earthkit.regrid import interpolate
import earthkit.plots.quickmap as qmap

In [ ]:
VAR_OF_INTEREST = '2t'

In [ ]:
# the target grid is a global 5x5 degree regular latitude grid
out_grid = {"grid": [0.25,0.25]}
r = interpolate(combined_data.sel(param = VAR_OF_INTEREST), out_grid=out_grid, method="linear")

In [ ]:
qmap.plot(r, colors = 'viridis')